In [1]:
import pandas as pd
from web_scrapping import web_scrape_wikipedia, web_scrape_seattle_children

In [2]:
drug_data = pd.read_csv("/Users/shobanasiranjeevilu/Downloads/Healthcare_LLM/drug_data.csv")

In [3]:
diseases = list(drug_data.Prescribed_for.dropna().unique())

In [4]:
diseases_list = []

In [5]:
# cleaning the unnecesary disease type is yet to be processed
disease_list = []
for disease in diseases:
  if "</span>" not in disease:
    disease_list.append(disease)


In [6]:
#  only considering drugs which has medium to high rating
filtered_df = drug_data[(drug_data['Prescribed_for'].isin(disease_list)) & (drug_data['User_Rating']>=5)]

In [7]:
aggregated_df = filtered_df.groupby('Prescribed_for').agg(DrugName_list=('drugName', 'unique')).reset_index(drop=False)

In [8]:
diseases_final = list(aggregated_df['Prescribed_for'])

### Web scraping to retrive symptoms from wikipedia

In [9]:
disease_with_symptoms = web_scrape_wikipedia(diseases_final)

In [11]:
print("Number of diseases retrived from wikipedia is {}".format(len(disease_with_symptoms.keys())))

Number of diseases retrived from wikipedia is 502


In [12]:
# extracting diseases that's not been retrieved from wikipedia

unretrived_diseases = [key for key in diseases_final if key not in disease_with_symptoms.keys()]

In [13]:
print("Number of diseases unretrived from wikipedia is {}".format(len(unretrived_diseases)))

Number of diseases unretrived from wikipedia is 132


### Web scraping to retrive remaining disease's symptoms from seattlechildren's

In [14]:
base_url = 'https://www.seattlechildrens.org'

unretrived_diseases_symptoms = web_scrape_seattle_children(base_url, unretrived_diseases)

In [16]:
final_disease_symptoms = {**disease_with_symptoms, **unretrived_diseases_symptoms}

In [21]:
print("Final number of diseases available with symptoms {}".format(len(final_disease_symptoms.keys())))

Final number of diseases available with symptoms 504


### creating final dataframe with all the disease's and symptoms with drug list

In [22]:
prescribed_for, symptoms = [], []
for key, value in final_disease_symptoms.items():
    prescribed_for.append(key)
    symptoms.append(value)


In [24]:
symptoms_df = pd.DataFrame({'Prescribed_for': prescribed_for, 'Symptoms': symptoms}, index=range(1, len(final_disease_symptoms) + 1))

In [25]:
final_df = aggregated_df.merge(symptoms_df,on='Prescribed_for', how='left')

In [27]:
final_df.head(5)

,Prescribed_for,DrugName_list,Symptoms
0,ADHD,"[Clonidine, Bupropion, Vyvanse, Dexmethylpheni...",Inattention carelessness hyperactivity executi...
1,AIDS Related Wasting,[Serostim],Early : Flu-like illness Later : Large lymph n...
2,Abnormal Uterine Bleeding,"[Ethinyl estradiol / levonorgestrel, Mirena, L...","Irregular, abnormally frequent, prolonged, or ..."
3,Abortion,[Misoprostol],NaN
4,Acetaminophen Overdose,[Mucomyst-10],"Early : Non specific, feeling tired, abdominal..."
